1. Load directory

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents(directory):
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    print(f"Loaded {len(documents)} documents")
    return documents

documents = load_documents("data")

2. Chunk documents

In [ ]:
from typing import List, Dict, Any
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents: List[Dict[str, Any]], chunk_size: int = 1000, chunk_overlap: int = 200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunk_docs = text_splitter.split_documents(documents)
    print(f"Split into {len(chunk_docs)} chunks")
    return chunk_docs

chunk_docs = split_documents(documents)

3. Setup vector store

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI, OpenAI
from pinecone import Pinecone
import os

def setup_vector_store(chunk_docs: List[Dict[str, Any]], index_name: str) -> PineconeVectorStore:
    openai_api_key = os.getenv("OPENAI_API_KEY")
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
    pc = Pinecone(api_key=pinecone_api_key)
    
    vector_store = PineconeVectorStore.from_documents(
        chunk_docs,
        embeddings,
        index_name=index_name
    )
    return vector_store

vector_store = setup_vector_store(chunk_docs, "qo-cc")

4. Setup and test "simple retriever"

In [ ]:
def setup_simple_retriever(query):
    return vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

query = "What is COMPACT in RAG?"

retriever = setup_simple_retriever(query)
retrieved_docs = retriever.invoke(query)

for idx, doc in enumerate(retrieved_docs, start=1):
    print(f"Document {idx}: \n{doc.page_content}")

5. Query Optimization

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

def optimize_query_with_context(query: str, vector_store: PineconeVectorStore):
    # First, get relevant documents from the vector store
    base_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    relevant_docs = base_retriever.invoke(query)
    
    # Extract content from relevant documents
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Create a prompt that includes the context
    optimization_template = """Given the following context and user query, optimize the query for retrieval. 
    Focus on key concepts that are present in both the query and the context. 
    Expand on these concepts using terminology from the context to improve retrieval effectiveness.
    Do not provide an answer to the query, only optimize it for better retrieval.

    Context:
    {context}

    User query: {query}
    
    Optimized query:"""
    
    optimization_prompt = PromptTemplate(
        template=optimization_template, 
        input_variables=['context', 'query']
    )
    
    llm = OpenAI()  
    
    # Use the pipe operator to create the chain
    optimization_chain = optimization_prompt | llm
    
    # Generate the optimized query
    optimized_query = optimization_chain.invoke({"context": context, "query": query})
    
    return optimized_query.strip()

optimized_query = optimize_query_with_context(query, vector_store)
print(f"Original query: {query}")
print(f"\nOptimized query: {optimized_query}")

6. Setup Compression Retriever

In [ ]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

def setup_compression_retriever(vector_store: PineconeVectorStore) -> ContextualCompressionRetriever:
    base_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
    llm = ChatOpenAI(temperature=0)
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=base_retriever
    )
    return compression_retriever

compression_retriever = setup_compression_retriever(vector_store)

7. Test Compression Retriever

In [ ]:
compressed_docs = compression_retriever.invoke(query)
print("Compressed Retrieval Results:")
for i, doc in enumerate(compressed_docs, 1):
    print(f"\nDocument {i}:")
    print(doc.page_content)  

8. Setup QA chain

In [ ]:
from langchain.chains import RetrievalQA

def setup_qa_chain(compression_retriever: ContextualCompressionRetriever) -> RetrievalQA:
    llm = ChatOpenAI(temperature=0)
    
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=compression_retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template="""Use the following pieces of context to answer the question at the end.
                If you don't know the answer, just say that you don't know. Do not try to make up an answer.
                
                {context}
                
                Question: {question}
                Answer:""",
                input_variables=['context', 'question']
            ),
        },
        verbose=True
    )

qa_chain = setup_qa_chain(compression_retriever)


9. Run QA chain

In [ ]:
result = qa_chain({'query': optimized_query})

print(f"Original query: {query}")
print(f"Oprimised query: {optimized_query}")
print("\nFinal Answer:")
print(result['result'])

print("\nSource Documents:")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"\nSource Document {i}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")